In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_experiments.library.tomography import ProcessTomography
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeKyoto
from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import Chi, Choi, Kraus, SuperOp
import numpy as np
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment

In [2]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_osaka")

In [2]:
# simulator = AerSimulator.from_backend(FakeKyoto())
# ns_simulator = AerSimulator.from_backend(GenericBackendV2(num_qubits=2))

In [3]:
# def qspt(qc, backend, max_circuits, qubits_used, analysis='default', backend_type='simulator'):
#     if backend_type == 'real':
#         backend.open_session()
#     exp = ProcessTomography(qc, backend, physical_qubits=qubits_used, analysis=analysis)
#     exp.set_experiment_options(max_circuits=max_circuits)
#     exp_data = exp.run(backend=backend, analysis=analysis)
#     if backend_type == 'real':
#         backend.close_session()
#     return exp_data

# def loop_qspt(qc_ls, backend, qubits_used, analysis='default', backend_type='simulator', max_circuits=300):
#     exp_data_ls = []
#     for circuit in qc_ls:
#         exp_data_ls.append(qspt(circuit, backend, max_circuits, qubits_used, analysis, backend_type))
#     return exp_data_ls

def generate_circ_ls(base_qc, num_partitions):
    qc_ls = []
    # print(base_qc.draw())
    for i in range(num_partitions):
        curr_qc = base_qc.repeat(i)
        curr_qc = curr_qc.decompose(reps=1)
        qc_ls.append(curr_qc)
    return qc_ls

def to_matrix(l, n):
    arr_ls = [l[i:i+n] for i in range(0, len(l), n)]
    ls = [l.tolist() for l in arr_ls]
    return ls

def exp_2q(qc_ls, backend, qubit_ls, analysis='default', max_circuits=300):
    exp_ls = []
    for i in range(len(qc_ls)):
          curr_qc = qc_ls[i]
          curr_qubits_used = qubit_ls[i]
          curr_exp = ProcessTomography(curr_qc, backend, physical_qubits=curr_qubits_used, analysis=analysis)
          exp_ls.append(curr_exp)
        #   parallel_exp_ls = ParallelExperiment(exp_ls[0: 63], flatten_results=False)
    parallel_exp_ls = [ParallelExperiment(exp_ls[0: 63], flatten_results=False), ParallelExperiment(exp_ls[63: len(exp_ls)], flatten_results=False)]
    batch_exp = BatchExperiment(parallel_exp_ls, flatten_results=False)
    return batch_exp
                       


# def post_process(exp_data_ls):
#     return measure



In [13]:
id_circ = QuantumCircuit(2)
id_circ.id(0)
id_circ.id(1)

circ_ls = generate_circ_ls(id_circ, 126)
# circ_ls[4].draw()
qubit_ls = to_matrix(np.arange(0, 126), 2)
qubit_ls.extend(qubit_ls)
exp = exp_2q(circ_ls, backend=backend, qubit_ls=qubit_ls)
# qubit_ls
q2_data = exp.run(backend=backend)
q2_data.auto_save = True

In [6]:
ProcessTomography(id_circ, backend=backend, physical_qubits=[1, 2], analysis='default').run()

ExperimentData(ProcessTomography, d8b0bcbf-6128-4477-94ef-5df32990ac96, job_ids=['crqxdne3qnq000858afg'], metadata=<5 items>)

Job was cancelled before completion [Job ID: crqxdne3qnq000858afg]


In [191]:
qc = QuantumCircuit(1)
qc.x(0)
qc.num_qubits
qc_ls = generate_circ_ls(base_qc=qc, num_partitions=5)
qc_ls[3].draw()
# exp_data = batch_qspt(qc_ls, backend=ns_simulator, qubits_used = range(0, 1))

In [192]:
exp_data[3].analysis_results(dataframe=True)

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
aec0e6e5,state,ProcessTomography,[Q0],"Choi([[ 0.00456883+0.00000000e+00j, 0.0011961...",None,aer_simulator_from(generic_backend_2q),None,2.0,"[1.972256651480704, 0.027743348519291368, 0.0,...","[1.9826475990404004, 0.038134296078987724, -9....",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
c7ecee77,process_fidelity,ProcessTomography,[Q0],0.986017,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,None,None
629da5e3,completely_positive,ProcessTomography,[Q0],True,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,None,None
324e41e9,trace_preserving,ProcessTomography,[Q0],False,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,0.012197,"(None, None)"


In [190]:
result = qspt(qc, ns_simulator, 300, qubits_used=range(0, 1))

In [188]:
a = result.analysis_results(dataframe=True)
a

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
1eaa871a,state,ProcessTomography,[Q0],"Choi([[ 3.03255316e-03+0.00000000e+00j, 1.215...",None,aer_simulator_from(generic_backend_2q),None,2.0,"[1.9856119690528715, 0.01438803094712366, 0.0,...","[2.0022366297934924, 0.031012691687744544, 0.0...",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
43b3c0dd,process_fidelity,ProcessTomography,[Q0],0.992665,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,None,None
66f5f770,completely_positive,ProcessTomography,[Q0],True,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,None,None
30352149,trace_preserving,ProcessTomography,[Q0],False,None,aer_simulator_from(generic_backend_2q),None,None,None,None,None,None,None,None,0.035065,"(None, None)"


In [72]:
value = a.iloc[0].value
eigvals = a.iloc[0].eigvals

In [73]:
value

Choi([[ 0.00606083+0.00000000e+00j, -0.01579157+9.30501208e-03j,
       -0.0113328 +1.11067465e-02j,  0.00297409-8.19291737e-04j],
      [-0.01579157-9.30501208e-03j,  0.99477263-1.96249408e-17j,
        0.9930149 +9.12054196e-03j,  0.00773035-1.01471958e-02j],
      [-0.0113328 -1.11067465e-02j,  0.9930149 -9.12054196e-03j,
        0.99659618+2.31094302e-17j,  0.0096521 -1.12555197e-02j],
      [ 0.00297409+8.19291737e-04j,  0.00773035+1.01471958e-02j,
        0.0096521 +1.12555197e-02j,  0.00257036-3.02064724e-20j]],
     input_dims=(2,), output_dims=(2,))

In [74]:
eigvals

array([1.98922287e+00, 9.71304880e-03, 1.06408044e-03, 0.00000000e+00])

In [76]:
Chi(value)

Chi([[ 7.28968566e-03-5.42101086e-20j, -4.87095817e-03+2.09072371e-02j,
       1.45502922e-03-3.19026282e-03j,  1.74523277e-03+8.19291737e-04j],
     [-4.87095817e-03-2.09072371e-02j,  1.98869931e+00+6.93889390e-18j,
       9.12054196e-03-9.11771340e-04j, -2.22534105e-02+4.95478443e-04j],
     [ 1.45502922e-03+3.19026282e-03j,  9.12054196e-03+9.11771340e-04j,
       2.66950365e-03-3.46944695e-18j,  3.46705247e-04+1.26850749e-03j],
     [ 1.74523277e-03-8.19291737e-04j, -2.22534105e-02-4.95478443e-04j,
       3.46705247e-04-1.26850749e-03j,  1.34150492e-03+5.42101086e-20j]],
    input_dims=(2,), output_dims=(2,))

In [79]:
Choi(Chi(value)) == value

True

In [81]:
Kraus(value)

Kraus([[[-1.70525008e-02-5.27408668e-18j,
          7.99321655e-01+5.96959997e-01j],
        [ 7.93108518e-01+6.03691429e-01j,
          4.78470716e-04+1.38218949e-02j]],

       [[-7.55890625e-02-2.84574618e-16j,
         -3.12209807e-02+1.10308619e-02j],
        [ 3.08080234e-02-1.18710785e-02j,
         -4.08389795e-02-1.20429990e-02j]],

       [[ 7.50559159e-03-7.16641231e-11j,
         -8.30456614e-03-1.24271640e-02j],
        [ 8.21570844e-03+1.22725593e-02j,
         -1.39536879e-02+1.92750443e-02j]]],
      input_dims=(2,), output_dims=(2,))

In [84]:
SuperOp(value)

SuperOp([[ 0.00606083+0.00000000e+00j, -0.0113328 -1.11067465e-02j,
          -0.0113328 +1.11067465e-02j,  0.99659618+2.31094302e-17j],
         [-0.01579157-9.30501208e-03j,  0.00297409+8.19291737e-04j,
           0.9930149 +9.12054196e-03j,  0.0096521 +1.12555197e-02j],
         [-0.01579157+9.30501208e-03j,  0.9930149 -9.12054196e-03j,
           0.00297409-8.19291737e-04j,  0.0096521 -1.12555197e-02j],
         [ 0.99477263-1.96249408e-17j,  0.00773035+1.01471958e-02j,
           0.00773035-1.01471958e-02j,  0.00257036-3.02064724e-20j]],
        input_dims=(2,), output_dims=(2,))